In [1]:
cd ..

/home/dwright/repos/papers/Hu et al 2017


In [2]:
import os
import sys
sys.path.append('./exp-refgoog/refgoog-dataset/google_refexp_py_lib/')
from refexp import Refexp

import tqdm
import numpy as np

import util.io

In [3]:
# Specify datasets path.
refexp_filename_trn='./exp-refgoog/refgoog-dataset/google_refexp_dataset_release/google_refexp_train_201511_coco_aligned.json'
refexp_filename_val='./exp-refgoog/refgoog-dataset/google_refexp_dataset_release/google_refexp_val_201511_coco_aligned.json'
coco_filename='./exp-refgoog/coco-dataset/annotations/instances_train2014.json'
val_filename='./exp-refgoog/coco-dataset/annotations/instances_val2014.json'
image_dir = '/hdd/dustin/data/coco/images/train2014/'
val_image_dir = '/hdd/dustin/data/coco/images/val2014/'

# Create Refexp instance.
refexp_trn = Refexp(refexp_filename_trn, coco_filename)
refexp_val = Refexp(refexp_filename_val, coco_filename)

loading annotations into memory...
Done (t=18.65s)
creating index...
index created!
Dataset loaded.
loading annotations into memory...
Done (t=20.70s)
creating index...
index created!
Dataset loaded.


In [6]:
# imdb_format:
#   a list of images
# each image:
#   a dict
#   { "image_path": "...",
#     "regions": [[[x1, y1, x2, y2], (sentence1, sentence2)], ...],
#     "proposals": [[x1, y1, x2, y2], ...],
#     "misc": {"dataset": "coco", ...} }

def build_imdb_from_refexp(refexp, image_dir, n_k=5):
    imdb = []

    imgIds = refexp.getImgIds()
    for n_img_id in tqdm.trange(len(imgIds)):
        img_id = imgIds[n_img_id]
        img = refexp.loadImgs(img_id)[0]
        im_path = os.path.join(image_dir, img['file_name'])
        
        proposals = np.array([_convert_bbox(c['bounding_box']) for c in img['region_candidates']])
        
        
        regions, categories = _load_refexp_regions(refexp, img_id)
        
        #Negative samples, select out of category images
        cat_ids = [id_ for id_ in refexp.catIds if id_ not in categories]
        cat_choices = np.random.choice(len(cat_ids), n_k)
        neg_paths = []
        neg_proposals = []
        for i in range(n_k):
            sample_img_ids = refexp.getImgIds(catIds=cat_ids[cat_choices[i]])
            sample_id = sample_img_ids[np.random.choice(len(sample_img_ids))]
            
            ann_ids_coco = refexp.coco.getAnnIds(sample_id)
            anns_coco = refexp.coco.loadAnns(ann_ids_coco)
            
            neg_paths.append(os.path.join(image_dir, refexp.loadImgs(sample_id)[0]['file_name']))
            neg_proposals.append(np.asarray([_convert_bbox(ann['bbox']) for ann in anns_coco]))
        
        misc = {"dataset": "coco-G-Ref", "img_id": img_id}
        iminfo = {"im_path": im_path, "regions": regions, "proposals": proposals,
                  "neg_sample": {"im_path": neg_paths, "proposals": neg_proposals}, "misc": misc}

        imdb.append(iminfo)

    return imdb

def _convert_bbox(bbox_xywh):
    x, y, w, h = bbox_xywh
    return [x, y, x+w-1, y+h-1]

def _load_refexp_regions(refexp, img_id):
    ann_ids_gref = refexp.getAnnIds(img_id)
    anns_gref = refexp.loadAnns(ann_ids_gref)
    ann_ids_coco = refexp.coco.getAnnIds(img_id)
    anns_coco = refexp.coco.loadAnns(ann_ids_coco)
    
    categories = [ann['category_id'] for ann in anns_gref]
    
    # Align refexp annotations to those in coco
    anns_gref_map = {ann_ids_gref[n]: anns_gref[n] for n in range(len(ann_ids_gref))}
    for n in range(len(ann_ids_coco)):
        if ann_ids_coco[n] in anns_gref_map:
            anns_coco[n]['refexp_ids'] = anns_gref_map[ann_ids_coco[n]]['refexp_ids']
        else:
            anns_coco[n]['refexp_ids'] = []
    
    # Collect all the ground-truth object bounding boxes from COCO
    # Add language annotations to the bounding boxes
    regions = [[_convert_bbox(ann['bbox']),
                _load_refexp_sentences(refexp, ann['refexp_ids']),
                ann['id']] for ann in anns_coco]
    return regions, categories

def _load_refexp_sentences(refexp, refexp_ids):
    exps = [e['raw'] for e in refexp.loadRefexps(refexp_ids)]
    return exps

In [7]:
imdb_trn = build_imdb_from_refexp(refexp_trn, image_dir)
imdb_val = build_imdb_from_refexp(refexp_val, image_dir, n_k=0)

100%|██████████| 4650/4650 [00:03<00:00, 1229.29it/s]


In [8]:
if not os.path.isdir('./exp-refgoog/data/imdb'):
    os.mkdir('./exp-refgoog/data/imdb')
    
util.io.save_json(imdb_trn, '/home/dwright/code/Research/compositional_image_retrieval/exp-googleref/data/imdb/imdb_trn.json')
util.io.save_json(imdb_val, '/home/dwright/code/Research/compositional_image_retrieval/exp-googleref/data/imdb/imdb_val.json')